In [117]:
import math
import requests
import datetime
from statistics import mean
import time
import sys

YEAR = 2020
WEEK = 6
POST_WEEK = 0
LI_RATIO = .1
L_SHARE = LI_RATIO
I_SHARE = 1 - LI_RATIO
TRANSFER_RATIO = 1
MARGIN_LOG = 7

In [118]:
#prev = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=regular')
#prev_post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=postseason')
reg = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=regular')
post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=postseason')
records = {}
games = reg.json() + post.json() #+ prev.json() + prev_post.json()

In [115]:
for game in games:
        
    away_team = game['away_team']
    home_team = game['home_team']
    
    # With this dataset, no conference listed means they are an FCS team. 
    # I treat all FCS teams as the same, likely crappy team. 
    if not game['away_conference']:
        away_team = 'FCS Team'
    if not game['home_conference']:
        home_team = 'FCS Team'   
    
    if away_team not in records:
        records[away_team] = []
    if home_team not in records:
        records[home_team] = []
    
    if ((game['season_type'] != 'postseason'and game['week'] > WEEK) 
        or (game['season_type'] == 'postseason' and game['week'] > POST_WEEK)):
        continue
    
    # For invalid/unplayed games
    if game['away_points'] == None:
        continue
        
    margin = game['away_points'] - game['home_points']
    
    # Going to overtime counts as a one point win
    if len(game['away_line_scores']) > 4:
        margin = margin / abs(margin)
    records[away_team].append((home_team, margin))
    records[home_team].append((away_team, -margin))

In [107]:
records['Clemson']

[('Wake Forest', 24),
 ('FCS Team', 49),
 ('Virginia', 18),
 ('Miami', 25),
 ('Georgia Tech', 66),
 ('Syracuse', 26),
 ('Boston College', 6)]

In [116]:
start = time.time()
batch_start = time.time()

rankings = {}
scores = {}
for team in records:
    rankings[team] = 0
    scores[team] = [0]

team_count = len(rankings.keys())

rank_history = [] # used for checking if we have hit a repeating pattern based on ranks
past_averaged_history = '' # used to see if the patterns are broadly repeating
rank_history_set = set() 
score_history = [] # used for easier lookups of what team scores were in said repeating pattern
averaged_score_history = [] 
scores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)
repeat_point = 0
pattern_found = False
# Repeat for 100000 interations, but it should hit a repeating steady state before then
for i in range(1000001):
    for team in records.keys():
        scores[team] = [0] # the first value is the total score
        games = records[team]
        for game in games:
            opponent = game[0]
            margin = game[1]
            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1
            multiplier *= 1000 # just for nicer numbers
            game_score = 0
            if margin < 0:
                game_score -= L_SHARE * rankings[opponent] / team_count
                game_score -= I_SHARE / (team_count - rankings[opponent])
                if rankings[opponent] > rankings[team]:
                    game_score -= TRANSFER_RATIO * L_SHARE * (team_count - rankings[team]) / team_count
                    game_score -= TRANSFER_RATIO * I_SHARE / (1 + rankings[team])
            if margin > 0:
                game_score += L_SHARE * (team_count - rankings[opponent]) / team_count
                game_score += I_SHARE / (1 + rankings[opponent])
                if rankings[opponent] < rankings[team]:
                    game_score += TRANSFER_RATIO * L_SHARE * rankings[team] / team_count
                    game_score += TRANSFER_RATIO * I_SHARE / (team_count - rankings[team])
            scores[team] += [multiplier * game_score]
        if i == 0:
            scores[team][0] = sum(scores[team])
        elif i == 1:
            scores[team][0] = (sum(scores[team]) + score_history[-1][team][0]) / 2
        else:
            scores[team][0] = (sum(scores[team]) 
                               + (2 * score_history[-1][team][0] 
                                  - score_history[-2][team][0])) / 2
    # This is sorted, but adjusting so tied teams have the same (higher) rank
    new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
    #print(new_rank_orders)
    new_ranks = rankings
    new_ranks[new_rank_orders[0]] = 0
    past_rank = 0
    past_score = scores[new_rank_orders[0]][0]
    for rank, team in enumerate(new_rank_orders):
        if (abs(scores[team][0] - past_score) < 0.01):
            new_ranks[team] = past_rank
        else:
            new_ranks[team] = rank
        past_rank = new_ranks[team]
        past_score = scores[team][0]
    # stop if we are repeating ourselves, record this if we are not
    if str(new_ranks) in rank_history_set:
        repeat_point = rank_history.index(str(new_ranks))
        pattern_found = True
        break
    else:
        rank_history += [str(new_ranks)]
        rank_history_set.add(str(new_ranks))
        score_history += [scores.copy()]
        rankings = new_ranks
        
    # stop if we are *broadly* repeating ourselves
    if i%1000 == 0 and i != 0:
        print(i, 'Iterations Took: ' + str(round(time.time() - batch_start)))
        check_start = time.time()
        
        for team in records:
            score = [x[team] for x in score_history]
            scores[team] = list(map(mean, zip(*score)))
            scores[team][0] = sum(scores[team][1:]) 
        new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
        if str(new_rank_orders) == past_averaged_history:
            pattern_found = True
            break
        else:
            averaged_score_history += [scores.copy()]
            past_averaged_history = str(new_rank_orders)
        print(i, 'Check Took: ' + str(round(time.time() - check_start)))
        batch_start = time.time()
        
        
        
    if i>=1000:
        old_news = rank_history[0]
        rank_history_set.remove(old_news)
        del rank_history[0]
        del score_history[0]
            

print(i)

# average across repeated cycled
for team in records:
    score = []
    if pattern_found: 
        for i in range(repeat_point, len(rank_history)):
            score += [score_history[i][team]]
    else:
        score = [x[team] for x in averaged_score_history]
    scores[team] = list(map(mean, zip(*score)))
    scores[team][0] = sum(scores[team][1:]) 
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ('My Computer Poll!\n'
          + 'It only takes into account games played this season, looking at margin and quality of wins and losses individually:\n\n'
          + '|Rank|Score|Team|Breakdown|\n'
          + '|-|-|-|-|\n')
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(0, len(scores[team]) - 1):
        explanation += (str(round(scores[team][j + 1])) + ' from #'
                        + str(new_rank_orders.index(records[team][j][0]) + 1) + ' '
                        + str(records[team][j][0]) 
                        + '(' + str(records[team][j][1]) + '), ')
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + explanation[0:-2] + '\n')
    output += line
print('Completed in ' + str(time.time() - start) + ' seconds')
print(output)
with open('Output - '
        + str(YEAR) 
          + '-' 
          + str(WEEK) + '.md', 'w') as text_file:
    text_file.write(output)

1000 Iterations Took: 1
1000 Check Took: 1
1311
Completed in 2.893779754638672 seconds
My Computer Poll!
It only takes into account games played this season, looking at margin and quality of wins and losses individually:

|Rank|Score|Team|Breakdown|
|-|-|-|-|
|1|1211|Clemson(5-0)|259 from #36 Wake Forest(24), 23 from #128 FCS Team(49), 78 from #100 Virginia(18), 764 from #5 Miami(25), 86 from #105 Georgia Tech(66)
|2|1196|BYU(5-0)|316 from #32 Navy(52), 293 from #34 Troy(41), 256 from #39 Louisiana Tech(31), 68 from #98 UT San Antonio(7), 263 from #30 Houston(17)
|3|821|Alabama(4-0)|52 from #113 Missouri(19), 365 from #19 Texas A&M(28), 56 from #109 Ole Miss(15), 349 from #16 Georgia(17)
|4|779|Coastal Carolina(4-0)|56 from #111 Kansas(15), 20 from #128 FCS Team(22), 381 from #18 Arkansas State(29), 322 from #8 Louisiana(3)
|5|763|Miami(4-1)|315 from #21 UAB(17), 42 from #115 Louisville(13), 210 from #40 Florida State(42), -19 from #1 Clemson(-25), 215 from #38 Pittsburgh(12)
|6|636|Ok

In [103]:
ranks = []
for i in range (5):
    ranks.append(sorted(scores, key=averaged_score_history[-i].__getitem__, reverse=True))
for i in range(len(ranks[0])):
    row = str(i) + ' '
    for j in range (5):
        row += ranks[j][i] + ' '
    print(row)

0 LSU LSU LSU LSU LSU 
1 Ohio State Ohio State Ohio State Ohio State Ohio State 
2 Clemson Clemson Clemson Clemson Clemson 
3 Georgia Georgia Georgia Georgia Georgia 
4 Florida Florida Florida Florida Florida 
5 Oregon Oregon Oregon Oregon Oregon 
6 Penn State Penn State Penn State Penn State Penn State 
7 Alabama Wisconsin Wisconsin Wisconsin Wisconsin 
8 Wisconsin Alabama Alabama Alabama Alabama 
9 Oklahoma Oklahoma Oklahoma Oklahoma Oklahoma 
10 Notre Dame Notre Dame Notre Dame Notre Dame Notre Dame 
11 Memphis Memphis Memphis Memphis Memphis 
12 Auburn Auburn Auburn Auburn Auburn 
13 Baylor Baylor Baylor Baylor Baylor 
14 Minnesota Minnesota Minnesota Minnesota Minnesota 
15 Utah Utah Utah Utah Utah 
16 Cincinnati Cincinnati Cincinnati Cincinnati Cincinnati 
17 Navy Navy Navy Navy Navy 
18 Michigan Michigan Michigan Michigan Michigan 
19 Iowa Iowa Iowa Iowa Iowa 
20 Appalachian State Appalachian State Appalachian State Appalachian State Appalachian State 
21 Boise State USC USC USC